## Pipeline Nasıl Kullanılır?

- by: Bilalcan Ustabaş
- linkedin: https://www.linkedin.com/in/bilalcanustabas/

### 1. Gerekli Kütüphaneleri Yükleyelim

In [1]:
# Veri hazırlama kütüphaneleri ve fonksiyonları
import pandas as pd # DataFrame işlemleri için gerekli kütüphane
import numpy as np # Veri dönüşüm işlemleri için gerekli kütüphane
from sklearn.datasets import make_classification # Örnek veri oluşturmamızı sağlayan fonksiyon

# Veri ön işleme ve model otomatizasyonu sınıfları ve fonksiyonları
from sklearn.pipeline import Pipeline # Yazımızın konusu olan otomatizasyon sınıfı
from sklearn.model_selection import cross_val_score, train_test_split # Veri ayrıştırma ve cross validation yöntemleri için gerekli fonksiyonlar
from sklearn.metrics import f1_score, roc_auc_score # Model performanslarını karşılaştırmak için kullanılan metrik fonksiyonları
from sklearn.preprocessing import StandardScaler, MinMaxScaler # Preprocess işlemlerinde kullanılacak dönüşüm sınıfları

# Deneylerde kullanılacak model sınıfları
from sklearn.linear_model import LogisticRegression # Testler için kullanacağımız modellerden biri
from sklearn.ensemble import RandomForestClassifier # Testler için kullanacağımız modellerden biri
from lightgbm import LGBMClassifier # Testler için kullanacağımız modellerden biri
from catboost import CatBoostClassifier # Testler için kullanacağımız modellerden biri

# Deneylerde kullanılacak hiper parametre optimizasyon sınıfları ve kütüphaneleri
from sklearn.model_selection import GridSearchCV # Hiper parametre deneyleri için gerekli sınıf
import optuna # Hiper parametre deneyleri için gerekli kütüphane

### 2. Deney Hazırlığı

#### 2.1. Örnek Veri Oluşturulması

- 50000 satırlı %10 tahmin sınıf yüzdesine sahip 3 bilgi içeren 4 gereksiz 2 tekrarlı değişken içeren ve deneylerimizde kullanacağımız bir veri seti oluşturuyoruz.

In [2]:
# Oluşturulan verinin tekrar oluşturulabilmesi için seed belirlenmesi
np.random.seed(42)

# `make_classification` ile sınıflandırmada kullanılabilecek dengesiz hedef değişkenli 50000 satırlı verinin oluşturulması
X, y = make_classification(n_samples=50000, n_features=15, 
                           n_informative=3, n_redundant=4, n_repeated=2, 
                           weights=(0.9,0.1), shift=[np.random.lognormal(mean=1) for _ in range(15)],
                           random_state=42)

# Dataframe olarak incelemek için dönüşüm ve kolon isimlendirmesi
X = pd.DataFrame(X, columns=[f"col_{i}" for i in range(X.shape[1])])

# Oluşturduğumuz verinin detaylarının basılması
print("="*25, "   X DataFrame  ", "="*25)
display(X.head())
print("="*25, "Target Imbalance", "="*25)
print(f"0 count: {(y==0).sum()}")
print(f"1 count: {(y==1).sum()}")

# İşlem rahatlığı için dataframe'in numpy array'ine dönüştürülmesi
X = X.values

=========================    X DataFrame   =========================


,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,col_10,col_11,col_12,col_13,col_14
0,4.317765,12.194548,5.107623,2.196658,4.691893,4.859987,2.511852,-0.861836,0.008581,1.364854,13.373359,3.219474,3.094977,5.901621,0.839221
1,4.615254,13.178920,3.328998,1.748796,3.392489,6.321384,2.163355,1.332167,-0.993371,2.826251,13.357216,1.223225,2.438360,5.453758,3.675864
2,5.103577,14.962391,4.841555,0.959526,1.047548,8.960307,1.462348,-0.529126,0.730165,5.465174,13.278040,-2.487800,1.606609,4.664488,1.561553
3,5.379041,13.109531,2.999548,2.101685,4.638820,5.712265,0.973478,0.034586,-0.878587,2.217133,12.565993,0.299155,1.996482,5.806647,0.558182
4,4.461640,12.113986,2.851236,2.692785,3.907561,4.959464,1.315312,0.004511,0.115034,1.464331,12.412245,1.108321,1.555916,6.397747,2.320654


========================= Target Imbalance =========================
0 count: 44794
1 count: 5206


#### 2.2. Train-Test-Split İşlemlerinin Gerçekleştirilmesi

In [3]:
# Eğitim ve test kümelerinin performanslarını ayrı ayrı incelemek için bölme işlemini gerçekleştiriyoruz
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### 3. Basic Pipeline Kurulumu ve Çalıştırılması

- Pipeline sınıfı içerisinde veri dönüştüren sınıflar ve model tahmini gerçekleştirecek estimator'lar barındırabilir.
- Pipeline içerisine yazılan Model içerisine verilebilen model parametreleri iletebiliriz. 
- Veri dönüşümü gerçekleştiren sınıfların tahmin gerçekleştiren estimator sınıflardından önce gelmesi gerekmektedir.
- Tüm sınıfların kendi sınıf tiplerine göre içerilerinde belli fonksiyonları bulundurmak zorundadır. 
- Yapıya uyum sağlamayan sınıflar Pipeline'nın hata almasına sebep verirler.
- İlerleyen bölümlerde sınıfların hangi fonksiyonları içermesi gerektiğini göreceğiz.

#### 3.1. Pipeline Kurulumu

In [4]:
# Verilerimizi eğitim aşamasında kullanmadan önce yapmak istediğimiz basit bir ön işlem örneği olarak standardizasyon kullanalım
# Model olarak LogisticRegression ile deneyimizi gerçekleştirelim
pipe = Pipeline([
        ('preprocesser', StandardScaler()), 
        ('estimator', LogisticRegression())
        ])

# Model .fit() edermiş gibi Pipeline sınıfı da içerisindeki aşamalarda bulunan .fit() fonksiyonlarını sırasıyla kullanır
pipe.fit(X_train, y_train)

Pipeline(steps=[('preprocesser', StandardScaler()),
                ('estimator', LogisticRegression())])

#### 3.2. Pipeline ile Test Seti Üzerinde Veri Dönüşümü Gerçekleştirmek

- .fit() ile fit edilmiş bir Pipeline objesinin .transform() fonksiyonunu kullanarak eğitim seti haricindeki veri setleri de dönüştürülebilir

In [5]:
X_test_transformed = pipe.named_steps['preprocesser'].transform(X_test)

#### 3.3. Pipeline ile Tahmin Oluşturmak ve Değerlendirmek

- Tüm makine öğrenmesi modellerinde standartlaşmış .predict() ve .predict_proba() fonskiyonlarını Pipeline sınıfı ile kullanabiliyoruz. Bu sayede veri tahminlerini üretip değerlendirebiliriz.
- .predict() ve .predict_proba() içerisine verilen verileri Pipwlinw boyu uygulanmış preprocess adımlarındaki tranformation'ları uygular, ekstra işleme gerek duyulmaz.

In [6]:
# Tahminlemede kullanılacak bir fonksiyon oluşturalım
def pipe_score(p, X_train, y_train, X_test, y_test):

    # 0-1 tahminlerinin oluşturulması
    y_pred_train = p.predict(X_train)
    y_pred_test = p.predict(X_test)

    # Tahmin proba'ların oluşturulması
    y_pred_proba_train = p.predict_proba(X_train)[:,1]
    y_pred_proba_test = p.predict_proba(X_test)[:,1]

    # Metrik hesaplanması
    print("="*25, "     Metrics    ", "="*25)
    print(f"F1 Score: train: {f1_score(y_train, y_pred_train)}")
    print(f"F1 Score: test: {f1_score(y_test, y_pred_test)}")
    print(f"ROC-AUC Score: train: {roc_auc_score(y_train, y_pred_proba_train)}")
    print(f"ROC-AUC Score: test: {roc_auc_score(y_test, y_pred_proba_test)}")

In [7]:
# Pipeline sınfı içerisnde kullanılan model (estimator) sınıflarının .predict() ve .predict_proba() fonskiyonlarını kullanabilir
pipe_score(pipe, X_train, y_train, X_test, y_test)

=========================      Metrics     =========================
F1 Score: train: 0.804100858963702
F1 Score: test: 0.8028872848417546
ROC-AUC Score: train: 0.9110844241047773
ROC-AUC Score: test: 0.9073245318527652


#### 3.4. Pipeline ile Farklı Model Deneme Sürecinin Basitliği

- Pipeline'nın içerisinde bulunan 'estimator' diye adlandırdığımız kısma başka bir model objesini koyarak aynı süreci başka bir model için tekrarlayabiliriz.
- Pipeline içerisine yazılan Model içerisine verilebilen model parametreleri iletebiliriz.

In [8]:
# Model olarak RandomForestClassifier ile deneyimizi gerçekleştirelim
pipe_rf = Pipeline([
        ('preprocesser', StandardScaler()), 
        ('estimator', RandomForestClassifier(**{'random_state':42}))
        ])

pipe_rf.fit(X_train, y_train)

pipe_score(pipe_rf, X_train, y_train, X_test, y_test)

=========================      Metrics     =========================
F1 Score: train: 0.999879995199808
F1 Score: test: 0.8620689655172414
ROC-AUC Score: train: 0.999999993302799
ROC-AUC Score: test: 0.9611378748612183


In [9]:
# Model olarak LGBMClassifier ile deneyimizi gerçekleştirelim
pipe_lgbm = Pipeline([
        ('preprocesser', StandardScaler()), 
        ('estimator', LGBMClassifier(**{'random_state':42,
                                        'verbose':-100}))
        ])

pipe_lgbm.fit(X_train, y_train)

pipe_score(pipe_lgbm, X_train, y_train, X_test, y_test)

=========================      Metrics     =========================
F1 Score: train: 0.899756128866641
F1 Score: test: 0.8583333333333333
ROC-AUC Score: train: 0.9966410858370133
ROC-AUC Score: test: 0.9649890193619467


In [10]:
# Model olarak CatBoostClassifier ile deneyimizi gerçekleştirelim
pipe_cat = Pipeline([
        ('preprocesser', StandardScaler()), 
        ('estimator', CatBoostClassifier(**{'random_state':42,
                                            'verbose':0}))
        ])

pipe_cat.fit(X_train, y_train)

pipe_score(pipe_cat, X_train, y_train, X_test, y_test)

=========================      Metrics     =========================
F1 Score: train: 0.9149126164051538
F1 Score: test: 0.8535433070866143
ROC-AUC Score: train: 0.9912930895983488
ROC-AUC Score: test: 0.9638766168743843


### 4. Özelleştirilmiş Transformation Sınıflarının Yazılması

#### 4.1. Özelleştirilmiş Sınıfların Elemanları

- Özelleştirilmiş sınıfların, __init__, fit, transform fonksiyonlarını kesinlikle içermesi gerekir. Çünkü bu fonksiyonlar Pipeline'nın otomatizasyon sürücünde bulunmaktadır.
- inverse_transform, fit_transform gibi fonksiyonlar gerekli olmamakla birlikte eklenebilir.
- Bunlar dışındaki fonksiyonlar eklenebilir fakat otomatizasyon sürecine otomatik olarak dahil olmazlar.

In [11]:
# Bu özel sınıf kolonlara ayrı ayrı işlemler uygulayacak
class CustomPreprocesser():
    
    # __init__ fonksiyonunda fit aşamasında verilecek parametreleri self ile çağırabilmek için None olarak tanımlıyoruz.
    def __init__(self, X = None, ornek_parametre = None):
        
        self.X = X
        self.ornek_parametre = ornek_parametre
        
    # .fit() fonksiyonu, Pipeline.fit() çağırıldığında gerçekleşir, fit() fonksiyonu içine vereceğimiz ekstra parametreler burada **params'ın içerisinde gelecektir.
    ## Eklemek istediğiniz girdileri **params dictionary'sini döngü içinde aratarak elde edebilirsiniz.
    ### .fit() fonksiyonunda transformation öncesi hazırlanma işlemlerinin yapılması gerekmektedir.
    def fit(self, X, y = None, **params):

        param_keys = list(params.keys()) # params içerisine gelen diğer parametreleri okuyoruz
        for key in param_keys:
            if "ornek_parametre" in key: # eğer istediğimiz isim bulunuyorsa istenilen yere kaydediyoruz
                self.ornek_parametre = params[key]
                 # transformation işlemleri için fitting işlemi

        # return self kalmalı yoksa Pipeline yapısı hata veriyor
        return self
    
    # .predict(), .predict_proba() öncesinde gerçekleşen transformation işlemlerini barındırır
    ## .fit() içerisinde hazırlanmış veya hazırlanmamış tüm ön işlemler bu fonksiyon altında gerçekleşir
    def transform(self, X):
        
        # eğer 'ornek_parametre' parametremiz mevcutsa transform işleminin yapılması
        if self.ornek_parametre != None:
            X[:,2] = self.ornek_parametre # örnek transform işlemi
            
        # herhangi bir parametrenin mevcutluğuna bağlı transform işlemi
        ## burada istenilen tüm veri dönüşüm işlemleri gerçekleştirilebilir
        X = "Bazı veri işlemleri"

        return X

#### 4.2. Gerçek Transform İşlemlerine Sahip Örnek

In [12]:
# Bu özel sınıf kolonlara ayrı ayrı işlemler uygulayacak
class CustomPreprocesser():
    
    # __init__ fonksiyonunda fit aşamasında verilecek parametreleri self ile çağırabilmek için None olarak tanımlıyoruz.
    def __init__(self, X = None, scaler_standard = None, scaler_min_max = None):
        
        self.X = X
        self.scaler_standard = scaler_standard
        self.scaler_min_max = scaler_min_max
    
    # .fit() fonksiyonu, Pipeline.fit() çağırıldığında gerçekleşir, fit() fonksiyonu içine vereceğimiz ekstra parametreler burada **params'ın içerisinde gelecektir.
    ## Eklemek istediğiniz girdileri **params dictionary'sini döngü içinde aratarak elde edebilirsiniz.
    ### .fit() fonksiyonunda transformation öncesi hazırlanma işlemlerinin yapılması gerekmektedir.
    def fit(self, X, y = None, **params):

        param_keys = list(params.keys()) # params içerisine gelen diğer parametreleri okuyoruz
        for key in param_keys:
            if "scaler_standard" in key: # eğer istediğimiz isim bulunuyorsa istenilen yere kaydediyoruz
                self.scaler_standard = params[key]
                self.scaler_standard.fit(X[:,2].reshape(-1, 1)) # transformation işlemleri için fitting işlemi
            elif "scaler_min_max" in key: # eğer istediğimiz isim bulunuyorsa istenilen yere kaydediyoruz
                self.scaler_min_max = params[key]
                self.scaler_min_max.fit(X[:,1].reshape(-1, 1)) # transformation işlemleri için fitting işlemi
        
        # return self kalmalı yoksa Pipeline yapısı hata veriyor
        return self
    
    # .predict(), .predict_proba() öncesinde gerçekleşen transformation işlemlerini barındırır
    ## .fit() içerisinde hazırlanmış veya hazırlanmamış tüm ön işlemler bu fonksiyon altında gerçekleşir
    def transform(self, X):
        
        # eğer 'scaler_standard' parametremiz mevcutsa transform işleminin yapılması
        if self.scaler_standard != None:
            X[:,2] = self.scaler_standard.transform(X[:,2].reshape(-1, 1)).ravel()
            
        # eğer 'scaler_min_max' parametremiz mevcutsa transform işleminin yapılması
        if self.scaler_min_max != None:
            X[:,1] = self.scaler_min_max.transform(X[:,1].reshape(-1, 1)).ravel()

        # herhangi bir parametrenin mevcutluğuna bağlı transform işlemi
        X[:,0] = np.sqrt(X[:,0])

        return X

#### 4.3. Özel Oluşturulmuş ve Parametre İhtiyacı Olan Sınıfların Kullanımı

- Oluşturduğumuz CustomPreprocesser sınıfında bulunan Scaler özelliklerini ayrı ayrı kullanmamız için onları Pipeline'nın .fit() kısmında parametre olarak vermemiz gerekiyor.
- .fit() içine verilen parametreler şu şekilde yazılır: {Pipeline aşamasının ismi}__{parametre_ismi}={parametre_variable}

In [13]:
# Pipeline'nın kurulması
pipe = Pipeline([
        ('preprocesser', CustomPreprocesser()),
        ('estimator', RandomForestClassifier())
        ])

pipe.fit(X_train, y_train,
         preprocesser__scaler_standard=StandardScaler(), # Bu Pipeline örneği için preprocesser yukarıda CustomPreprocesser()'ın bulunduğu adımı işaret eder
         preprocesser__scaler_min_max=MinMaxScaler()) # CustomPreprocesser() içinde belirlediğimiz parametre ismini de Pipeline adım ismi sonrasında '__' ile birleştirerek yazıyoruz

pipe_score(pipe, X_train, y_train, X_test, y_test)

=========================      Metrics     =========================
F1 Score: train: 0.901684292988641
F1 Score: test: 0.8589341692789968
ROC-AUC Score: train: 0.9934806231324897
ROC-AUC Score: test: 0.9560995196917366


### 5. Pipeline ile Cross Validation

- Cross validation yöntemlerini kullanarak Pipeline'da belirlediğimiz tüm ön işlem ve model adımlarını train kümelerinde .fit() edip train ve validation kümelerinde .transform() yapmasını sağlayabiliyoruz.
- Pipeline, içerisine model veya estimator olan tüm cross validation yöntemlerinde kullanılabilir.
- Bu örnekte scikit-learn kütüphanesinin `cross_val_score` fonksiyonuna Pipeline objemizi vererek cross validation gerçekleştireceğiz.

In [14]:
# Pipeline'ımızı oluşturuyoruz
pipe = Pipeline([
        ('preprocesser', CustomPreprocesser()),
        ('estimator', LogisticRegression())
        ])

# Pipeline.fit() içerisine verdiğimiz parametreleri bir dictionary'e kaydedip daha sonra cross_val_score içerisinde kullanacağız
fit_params = {"preprocesser__scaler_standard":StandardScaler(),
              "preprocesser__scaler_min_max":MinMaxScaler()}

# Estimator kısmına Pipeline objemizi verip diğer parametreleri de belirleyerek cross validation sürecimizi gerçekleştiriyoruz
## fit_params parametresine yukarıda hazırladığımız ve Pipeline.fit() içerisinde kullanılacak parametreleri dictionary olarak veriyoruz
cross_val_scores = cross_val_score(estimator=pipe, X=X, y=y, cv=5, scoring="roc_auc", fit_params=fit_params)

# Cross-Validation sonuçları
print("validation kümelerindeki validation roc-auc değerleri:", cross_val_scores)
print(f"validation kümelerinde ortalama ve sapma roc-auc: mean:{np.mean(cross_val_scores)} std:{np.std(cross_val_scores)}")

validation kümelerindeki validation roc-auc değerleri: [0.9232233  0.89186312 0.90091718 0.92350809 0.91176835]
validation kümelerinde ortalama ve sapma roc-auc: mean:0.9102560083980812 std:0.012422296892597703


### 6. Pipeline ile Hiper Parametre Optimizasyonu

- Farklı Hiper Parametre optimizasyon yöntemleri farklı implementasyonlar gerektiriyor olsa da Pipeline objesinin kullanımı ya estimator yerine verilerek yapılıyor ya da estimator objesinin fit edildiği kısımda Pipeline fit edilerek gerçekleştiriliyor.

#### 6.1. GridSearch ve Pipeline ile Hiper Parametre Optimizasyonu

In [15]:
# Pipeline'ımızı önceden oluşturduğumuz gibi oluşturuyoruz
pipeline = Pipeline([
        ('preprocesser', CustomPreprocesser()),
        ('estimator', RandomForestClassifier(**{"random_state":42}))
        ])

# Parametre uzayımızı oluşturuyoruz, Hızlı bir örnek olması için az sayıda parametre ve arama uzayı yerleştirdim
param_grid = {
    'estimator__n_estimators': [50, 100],
    'estimator__max_depth': [10, 20],
}

# GridSearchCV'nin estimator objesine pipeline objemizi verip kalan parametreleri veriyoruz
grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=3, scoring='roc_auc')

# Pipeline'ımız ihtiyaç duyduğu parametreleri Pipeline ile oluşturduğumuz grid_search objesinin .fit() fonksiyony içerisinde veriyoruz
grid_search.fit(X, y, 
                preprocesser__scaler_standard=StandardScaler(), 
                preprocesser__scaler_min_max=MinMaxScaler())

# Hiper parametre optimizasyon sonuçları
print("En iyi Hiper Parametreler: ", grid_search.best_params_)
print("En iyi Skor: ", grid_search.best_score_)

En iyi Hiper Parametreler:  {'estimator__max_depth': 10, 'estimator__n_estimators': 100}
En iyi Skor:  0.962463402576237


#### 6.2. Optuna ve Pipeline ile Hiper Parametre Optimizasyonu

In [16]:
# Define the objective function for Optuna
def objective(trial):
    
    # Pipeline objemizi oluşturuyoruz
    ## Pipeline içerisinde estimator adımında bulunan modelimiz içerisine Optuna'nın trial.suggest formatlarıyla patametre uzayımızı ve ilgili parametreleri veriyoruz
    pipeline = Pipeline([
        ('preprocesser', CustomPreprocesser()),
        ('estimator', RandomForestClassifier(
            n_estimators=trial.suggest_int('n_estimators', 50, 100),
            max_depth=trial.suggest_int('max_depth', 10, 20),
            **{"random_state":42}))
        ])
    
    # Pipeline.fit() içerisine verdiğimiz parametreleri bir dictionary içerisine kaydedip daha sonra başka bir fonksiyon yardımı ile kullanacağız
    fit_params = {"preprocesser__scaler_standard":StandardScaler(),
                  "preprocesser__scaler_min_max":MinMaxScaler()}
    
    # Optuna'nın kendi içerisinde cross validation olmadığı için bu adımda daha önce örneğini yaptığımız cross_val_score kullanıyoruz
    ## Pipeline.fit() fonksiyonuna verdiğimiz parametreleri fit_params içerisinde iletiyoruz
    score = cross_val_score(pipeline, X, y, cv=3, scoring='roc_auc', fit_params=fit_params).mean()
    return score

# Optuna study'si oluşturup objective fonksiyonunda tanımladığımız Pipeline'ımızı optimize ediyoruz
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

# Hiper parametre optimizasyon sonuçları
print("Best parameters found: ", study.best_params)
print("Best score: ", study.best_value)

[I 2024-08-12 08:43:22,594] Trial 0 finished with value: 0.9610669317694911 and parameters: {'n_estimators': 81, 'max_depth': 17}. Best is trial 0 with value: 0.9610669317694911.
[I 2024-08-12 08:43:37,440] Trial 1 finished with value: 0.9625086335799332 and parameters: {'n_estimators': 69, 'max_depth': 13}. Best is trial 1 with value: 0.9625086335799332.
[I 2024-08-12 08:43:48,236] Trial 2 finished with value: 0.9621204407633593 and parameters: {'n_estimators': 62, 'max_depth': 10}. Best is trial 1 with value: 0.9625086335799332.
[I 2024-08-12 08:43:57,749] Trial 3 finished with value: 0.9619898665739092 and parameters: {'n_estimators': 54, 'max_depth': 10}. Best is trial 1 with value: 0.9625086335799332.
[I 2024-08-12 08:44:09,344] Trial 4 finished with value: 0.9615378590201438 and parameters: {'n_estimators': 50, 'max_depth': 15}. Best is trial 1 with value: 0.9625086335799332.
[I 2024-08-12 08:44:21,910] Trial 5 finished with value: 0.9613019306454103 and parameters: {'n_estimator

Best parameters found:  {'n_estimators': 51, 'max_depth': 13}
Best score:  0.9626487592632397
